In [16]:
import torchvision
from torchvision import models
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

import os, subprocess
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

In [17]:
for name in dir(torchvision.models):
    print(name)

TypeError: 'str' object is not callable

In [18]:
import torch
import torch.nn as nn

from tqdm import tqdm

# vit_b_16은 image size가 224 x 224 여야 한다.
model = models.vit_b_16(pretrained=True)
model

VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_a

In [4]:
model.heads.head.out_features=33

In [5]:
#parameter 수 확인
total_params = sum(p.numel() for p in model.parameters())
total_params

86567656

In [6]:
data_dir = "./data"
dir_lst = os.listdir(data_dir)
for dir in dir_lst:
    dir_path = os.listdir(f"./data/{dir}")

    for name in dir_path:
        if name == ".ipynb_checkpoints":
            print(name)
            os.rmdir(f"./data/{dir}/{name}")

In [19]:
IMG_TRANSFORM = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224,224), antialias = True),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),
    ])

In [20]:
train_imgfolder = ImageFolder(root="./data/train",
                              transform=IMG_TRANSFORM,
                              target_transform=None
                             )

test_imgfolder = ImageFolder(root="./data/test",
                              transform=IMG_TRANSFORM
                             )

In [28]:
Image.MAX_IMAGE_PIXELS = None

train_dataloader = DataLoader(dataset=train_imgfolder,
							  batch_size = 64,
                              num_workers=os.cpu_count(),
                              shuffle=True,
                              drop_last=True 
                             )

test_dataloader = DataLoader(dataset=test_imgfolder,
							  batch_size = 64,
                              num_workers=os.cpu_count(),
                              shuffle=True,
                              drop_last=True 
                            )

In [29]:
# model setting

EPOCH = 150
LEARNING_RATE = 1e-3 
DEVICE = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE, weight_decay = 0.001)
criterion = nn.CrossEntropyLoss()
print("Using device: ",DEVICE)

Using device:  cuda:3


In [30]:
def loss_fn(class_output, label, criterion):
    CE_loss = criterion(class_output, label)

    return CE_loss 

In [31]:
def train(train_loader, model, optimizer, criterion, epoch):
    model.train()
    correct = 0
    total = 0
    
    epoch_loss = []
    for index, (img_data,labels) in enumerate(train_loader):
        img_data, labels = img_data.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        
        outputs = model(img_data)
        loss = loss_fn(outputs, labels, criterion)
        
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
        
        
        loss.backward()
        optimizer.step()

        epoch_loss.append(loss.item())
        if index%100==0:
            print(f'train loss : {np.mean(epoch_loss):>7f}, [epoch:{epoch}, iter:{index}]')
            
    accuracy = correct / total
    print(f"TRAIN ACCURACY : {(100*accuracy):>0.1f}% [{correct}/{total}]")
    return np.mean(epoch_loss)

In [32]:
def test(test_loader, model, criterion, epoch):
    model.eval()

    epoch_loss = []
    correct = 0
    total = 0
    
    with torch.no_grad():
        for index, (img_data,labels) in enumerate(test_loader):
            img_data, labels = img_data.to(DEVICE), labels.to(DEVICE)
            outputs = model(img_data)
    
            loss = loss_fn(outputs, labels, criterion)

            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            
            epoch_loss.append(loss.item())
    
    accuracy = correct / total
    
    print(f"TEST ACCURACY : {(100*accuracy):>0.1f}% [{correct}/{total}]")
    return np.mean(epoch_loss)

In [33]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
train_loss_lst, test_loss_lst = [], []

model_version = 1


test_best_loss = 100
test_current_loss = 100
early_stop_threshold = 5
early_stop_trigger = 0

model = model.to(DEVICE)

for i in tqdm(range(EPOCH), desc = 'Train'):
    print(f"EPOCH {i+1} \n-------------------------")
    train_loss = train(train_dataloader, model, optimizer, criterion, i+1)
    test_loss = test(test_dataloader, model, criterion, i+1)

    train_loss_lst.append(train_loss)
    test_loss_lst.append(test_loss)

    if test_loss < test_best_loss :
        print("...MODEL SAVE...")
        test_best_loss = test_loss 
        
        if model_version == 0:
            torch.save(model.state_dict(),'contrastive_model_for_style_weight_v2_without_target_style.pth')
        else:
            torch.save(model.state_dict(),'contrastive_model_for_style_weight_vit_b_16.pth')
        
    if test_current_loss < test_loss:
        early_stop_trigger += 1 
    else:
        early_stop_trigger = 0 
    test_current_loss  = test_loss 

    print(f'\nEPOCH:{i+1} | train loss : {train_loss}, test loss : {test_loss}\n')
    
    if early_stop_trigger >= early_stop_threshold:
        break
        
print("DONE!")

Train:   0%|                                      | 0/150 [00:00<?, ?it/s]

EPOCH 1 
-------------------------


/home/qkrwnstj/anaconda3/envs/qkrwnstj/lib/python3.9/site-packages/PIL/TiffImagePlugin.py:858: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


train loss : 3.480208, [epoch:1, iter:0]
train loss : 3.429360, [epoch:1, iter:100]
train loss : 3.389463, [epoch:1, iter:200]


/home/qkrwnstj/anaconda3/envs/qkrwnstj/lib/python3.9/site-packages/PIL/TiffImagePlugin.py:858: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


...

train loss : 2.235019, [epoch:71, iter:200]
train loss : 2.235260, [epoch:71, iter:300]
train loss : 2.239297, [epoch:71, iter:400]
train loss : 2.239040, [epoch:71, iter:500]
train loss : 2.239329, [epoch:71, iter:600]
train loss : 2.233844, [epoch:71, iter:700]
train loss : 2.238528, [epoch:71, iter:800]


/home/qkrwnstj/anaconda3/envs/qkrwnstj/lib/python3.9/site-packages/PIL/TiffImagePlugin.py:858: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


train loss : 2.239032, [epoch:71, iter:900]
train loss : 2.235412, [epoch:71, iter:1000]
train loss : 2.235345, [epoch:71, iter:1100]
train loss : 2.234926, [epoch:71, iter:1200]
TRAIN ACCURACY : 34.8% [27900/80128]


Train:  47%|██████████▍           | 71/150 [19:34:45<21:43:31, 990.02s/it]

TEST ACCURACY : 23.1% [4107/17792]

EPOCH:71 | train loss : 2.236946036164372, test loss : 2.6807731733047704

EPOCH 72 
-------------------------
train loss : 2.372239, [epoch:72, iter:0]
train loss : 2.229299, [epoch:72, iter:100]
train loss : 2.233292, [epoch:72, iter:200]


In [ ]:
figure = plt.figure(figsize = (10,5))
rows, cols = 1, 2 

title = ['train loss', 'test loss']
values = [train_loss_lst, test_loss_lst]
for i in range(1, rows* cols +1):
    figure.add_subplot(rows,cols,i)
    plt.title(title[i-1])
    plt.plot(values[i-1])

plt.show()